Test and evaluate the LLM

In [1]:
%pip install torch
%pip install peft
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "./models/codeLlama-7b-text-to-sql" #"./models/codeLlama-7b-text-to-sql"
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/root/dev/genai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.43s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM',

In [10]:
from datasets import load_dataset
from random import randint

# Load our test dataset

# val_template = [{"content":"You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.","role":"system"},
#                 {"content":"Who is the youngest employee in the company? List employee's first and last name.","role":"user"},
#                 {"content":"SELECT first_name, last_name FROM employees ORDER BY birth_date DESC LIMIT 1","role":"assistant"}]


val_template = [{"content":"You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.","role":"system"},
                
                {"content":"SQL query: How many Top 10s have Wins larger than 0, and Poles larger than 0?","role":"user"},
                {"content":"SELECT COUNT(top_10) FROM table_name_35 WHERE wins > 0 AND poles > 0","role":"assistant"}]

# Test on sample

prompt = pipe.tokenizer.apply_chat_template(val_template, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Query:\n{val_template[1]['content']}")
print(f"Original Answer:\n{val_template[2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
SQL query: How many Top 10s have Wins larger than 0, and Poles larger than 0?
Original Answer:
SELECT COUNT(top_10) FROM table_name_35 WHERE wins > 0 AND poles > 0
Generated Answer:
How many Top 10s have Wins larger than 0, and Poles larger than 0?


In [ ]:

# from datasets import load_dataset
# from random import randint

# Load our test dataset

# eval_dataset = load_dataset("json", data_files="retrain/test_dataset.json", split="train")
# # rand_idx = randint(0, len(eval_dataset))


# for rand_idx in range(len(eval_dataset)):
#     # Test on sample
#     print(f"Test:\n{eval_dataset[rand_idx]['messages'][:2]}")

#     prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
#     outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    
#     print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
#     print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
#     print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")